In [1]:
# Importing

from gensim.models import Word2Vec
import os, sys, inspect
import numpy as np
import math
from Preprocessing.loadCleanly import sheets as sh
from gensim import corpora
from textmining import simple_tokenize_remove_stopwords
import lasagne
import theano.tensor as T
import theano
import copy
# Setting the data directories. 
# Preoprocessing.loadCleanly is what is what was being used already
_data_directory ='D:\GeneralInsights\Naytev\data'
_data_file = 'Annotated_Comments_for_Naytev_Facebook.csv'
_text_col = 0
_label_col = 5
header, rows = sh.get_spreadsheet_rows(os.path.join(_data_directory, _data_file), _text_col, dedupe=True)

#make the sentences and classes list. 
#Sentences contains sentences
#Classes contains the sentiment
sentences = [str(S[_text_col]) for S in rows]
classes   = [str(S[_label_col]) for S in rows]

#Tokeninzethe Sentence List
texts = [simple_tokenize_remove_stopwords(sentence) for sentence in sentences]
model = Word2Vec(texts, size=10, window=5, min_count=0, workers=2)



Processing csv input...
Detected csv comma delimiter...
Garbage removal and Deduping with parameter: True
Removed 88 duplicate comments.

In [2]:

# Construct the Sentiment LSTM Layer by Layer. The output of this will be a Lasagne Layer Object
def MakeSentimentLSTM(input_var):
    
    #batch size = 10 , 15 is the maximum sentence length, 10 is the word vector dimension
    l_in = lasagne.layers.InputLayer(shape=(10,15,10),
                                         input_var=input_var)
    l_lstm = lasagne.layers.LSTMLayer(l_in, num_units=10,peepholes=False)
    
    #l_pool = lasagne.layers.GlobalPoolLayer(l_lstm) # Not including the pooling layer for now

    l_shp =lasagne.layers.ReshapeLayer(l_in, (10*15, 10)) # the LSTM Layer must be reshaped See the example in the Docs
    
    l_out = lasagne.layers.DenseLayer(l_shp, num_units=10)
    
    return l_out


# Function to create random minibatches
# This won't be called until the LSTM is
# all Wired
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

# This converts the classes list to a list of digits
# Neural Networks tend to be better at classifying numbers
def ESentimentToDigits(Texts):
    Sentiments = []
    for Sentiment in Texts:
        if Sentiment == 'Positive':
            Sentiments.append(1)
        else:
            Sentiments.append(0)
    return np.array(Sentiments)

# The next 3 functions are set to pad a sentence with sequences of zeros if it is less than the maximum sentence length
# This is so the corpus has sequences that are all the same length

def PaddedZeros(InValue):
    return np.array([[0]*10]*InValue)

def GetMaxSentenceLength(InCorpus):
    MaxLength = 0
    for sentence in InCorpus:
        if len(sentence) > MaxLength:
            MaxLength = len(sentence)
    print MaxLength
    
def PadCorpus(InCorpus):
    MaxLength = GetMaxSentenceLength(InCorpus)
    Texts = []
    for sentence in InCorpus:
        SentenceOne = sentence+PaddedZeros(15-len(sentence)).tolist()
        Texts.append(SentenceOne)
    return Texts

In [3]:

# Prepare Theano variables for inputs and targets
input_var = T.tensor3('inputs')
target_var = T.ivector('targets')
# Create neural network model
network = MakeSentimentLSTM(input_var)

In [4]:
# get the output. This is a 1d layer as per the shape of the output DenseLayer
prediction = lasagne.layers.get_output(network)

# From the output and the known output, compute the loss function. 
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

C:\Users\ellmi_000\Anaconda\lib\site-packages\theano\scan_module\scan.py:1019: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)


In [5]:
# Collect up all of the shared variables
params = lasagne.layers.get_all_params(network, trainable=True)

# Feed the Loss Function Error and the shared variables into the update function
# This can be adadelta, Stoachastc Gradient Descent, etc. etc. or whatever you want
# But this is where the weights are updated 
updates = lasagne.updates.adadelta(
        loss, params)


In [6]:

# Wire up the training function to the training variables
# This inputs to this are empty theano variables not actual data
train_fn = theano.function([input_var, target_var], loss, updates=updates)

C:\Users\ellmi_000\Anaconda\lib\site-packages\theano\scan_module\scan_perform_ext.py:135: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [7]:
# Here is where I make the training data


# PadCorpus with zeroes 
# Also convert it to a numpy array

D = [[model[word].tolist() for word in sentence] for sentence in texts]
Texts = PadCorpus(D)
TextNP = np.array(Texts) 

# Make a numpy Results array from the array of sentiment
ResultsArray = ESentimentToDigits(classes)

# Create the minibatch
# For testing I'm only doing one
TextBatch = TextNP[0:10]
ResultsBatch = ResultsArray[0:10]

#Print Just because
print TextBatch.shape
print ResultsBatch.shape

15


In [19]:

# Train the network
# Notice this is where the actual data is fed into the network
# Figure this function out and you are good
train_fn(TextBatch,ResultsBatch)

IndexError: index 10 is out of bounds for axis 0 with size 10
Apply node that caused the error: CrossentropyCategorical1Hot(Elemwise{Composite{(i0 * (Abs(i1) + i2 + i3))}}[(0, 2)].0, targets)
Toposort index: 220
Inputs types: [TensorType(float64, matrix), TensorType(int32, vector)]
Inputs shapes: [(150L, 10L), (10L,)]
Inputs strides: [(80L, 8L), (4L,)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Sum{acc_dtype=float64}(CrossentropyCategorical1Hot.0), Shape_i{0}(CrossentropyCategorical1Hot.0)]]

Backtrace when the node is created:
  File "C:\Users\ellmi_000\Anaconda\lib\site-packages\lasagne\objectives.py", line 129, in categorical_crossentropy
    return theano.tensor.nnet.categorical_crossentropy(predictions, targets)

HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.